<a href="https://colab.research.google.com/github/IvanKatorgin/Deep-Learning/blob/main/Deep_Learning_%D0%B4%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Курс "Deep Learning". Домашнее задание 2. Многослойная нейронная сеть. Каторгин И.П.

## Задание

Постройте модель на основе полносвязных слоёв для классификации Fashion MNIST из библиотеки torchvision (https://docs.pytorch.org/vision/stable/datasets.html).

Получите качество на тестовой выборке не ниже 88%

Инструкция по выполнению задания
1. Скачайте тренировочную и тестовою часть датасета Fashion MNIST
2. Постройте модель, выбрав стартовую архитектуру
3. Обучите модель и сверьте качество на тестовой части с заданным порогом
4. Изменяйте архитектуру модели пока качество на тестовой части не будет выше порога. Вариации архитектуры можно реализовать через изменение количества слоёв, количества нейронов в слоях и использование регуляризации. Можно использовать различные оптимизаторы.

In [1]:
# загрузим библиотеки
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, time
%matplotlib inline

In [2]:
import torch
import torchvision as tv
from torch import autograd

In [3]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, random_split, DataLoader

Загрузим датасет

In [4]:
BATCH_SIZE=256

In [5]:
# загрузим датасет
train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

100%|██████████| 26.4M/26.4M [00:01<00:00, 19.2MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 343kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 6.20MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 8.78MB/s]


In [21]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

Создадим модель

In [22]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)

In [23]:
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=10, bias=True)
)

In [24]:
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.SGD(model.parameters(), lr=.01)
num_epochs = 15

In [25]:
def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()

        model.train()
        for X, y in train:
            trainer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)

        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)

        print("ep: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

Обучим модель

In [26]:
train_model()

ep: 0, taked: 8.969, train_loss: 1.7649857845712216, train_acc: 0.5409833333333334, test_loss: 1.29611876308918, test_acc: 0.6449
ep: 1, taked: 9.269, train_loss: 1.0756583916380051, train_acc: 0.67015, test_loss: 0.9448431253433227, test_acc: 0.6763
ep: 2, taked: 9.400, train_loss: 0.8635012449102198, train_acc: 0.7070666666666666, test_loss: 0.8185941740870476, test_acc: 0.7087
ep: 3, taked: 9.455, train_loss: 0.7701501087939485, train_acc: 0.7386333333333334, test_loss: 0.7488710880279541, test_acc: 0.7354
ep: 4, taked: 8.571, train_loss: 0.7124457970578619, train_acc: 0.7593666666666666, test_loss: 0.7011205449700355, test_acc: 0.7532
ep: 5, taked: 9.477, train_loss: 0.6706742786346598, train_acc: 0.77485, test_loss: 0.6652295432984829, test_acc: 0.769
ep: 6, taked: 9.353, train_loss: 0.6383450553772297, train_acc: 0.7875833333333333, test_loss: 0.6371999308466911, test_acc: 0.7808
ep: 7, taked: 9.434, train_loss: 0.6125058933775476, train_acc: 0.7967666666666666, test_loss: 0.6147

Заменим SGD на Adam и RMSProp

In [27]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)

In [28]:
trainer = torch.optim.Adam(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 9.815, train_loss: 0.5206882904184625, train_acc: 0.8111833333333334, test_loss: 0.43169169798493384, test_acc: 0.845
ep: 1, taked: 9.799, train_loss: 0.3852374642453295, train_acc: 0.8603666666666666, test_loss: 0.4052948631346226, test_acc: 0.8561
ep: 2, taked: 9.877, train_loss: 0.3481611762909179, train_acc: 0.87315, test_loss: 0.4181889433413744, test_acc: 0.8534
ep: 3, taked: 9.330, train_loss: 0.32977162244472097, train_acc: 0.8795666666666667, test_loss: 0.3993444390594959, test_acc: 0.8636
ep: 4, taked: 9.997, train_loss: 0.31558352055701805, train_acc: 0.8843, test_loss: 0.41095768585801123, test_acc: 0.8591
ep: 5, taked: 10.063, train_loss: 0.30580765617654676, train_acc: 0.8875833333333333, test_loss: 0.411845026910305, test_acc: 0.8612
ep: 6, taked: 10.004, train_loss: 0.29634868077775267, train_acc: 0.8915333333333333, test_loss: 0.4594195865094662, test_acc: 0.8527
ep: 7, taked: 9.811, train_loss: 0.28869172423443895, train_acc: 0.8929833333333334, test_los

In [29]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)

In [30]:
trainer = torch.optim.RMSprop(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 8.749, train_loss: 2.1278968349416205, train_acc: 0.7673333333333333, test_loss: 0.5489179141819477, test_acc: 0.7927
ep: 1, taked: 9.720, train_loss: 0.4621863995460754, train_acc: 0.8310333333333333, test_loss: 0.49561156183481214, test_acc: 0.8242
ep: 2, taked: 9.719, train_loss: 0.4201027499868515, train_acc: 0.8449833333333333, test_loss: 0.5590631410479545, test_acc: 0.8138
ep: 3, taked: 9.681, train_loss: 0.45981221382922316, train_acc: 0.8463833333333334, test_loss: 0.4323976539075375, test_acc: 0.8471
ep: 4, taked: 8.900, train_loss: 0.3679318416625895, train_acc: 0.8644166666666667, test_loss: 0.5056783333420753, test_acc: 0.8242
ep: 5, taked: 9.652, train_loss: 0.3595762860901812, train_acc: 0.8663666666666666, test_loss: 0.4219836715608835, test_acc: 0.8504
ep: 6, taked: 9.625, train_loss: 0.35856463129216054, train_acc: 0.8687833333333334, test_loss: 0.40507490187883377, test_acc: 0.8551
ep: 7, taked: 9.651, train_loss: 0.3435420477009834, train_acc: 0.874033

Сделаем больше слоёв и Adam

In [31]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10)
)

In [32]:
trainer = torch.optim.Adam(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 12.586, train_loss: 0.5813221334142887, train_acc: 0.78245, test_loss: 0.44911829456686975, test_acc: 0.8357
ep: 1, taked: 12.680, train_loss: 0.39604337240787263, train_acc: 0.8556166666666667, test_loss: 0.4276466406881809, test_acc: 0.8432
ep: 2, taked: 12.844, train_loss: 0.3649995906555906, train_acc: 0.86665, test_loss: 0.3866875607520342, test_acc: 0.8609
ep: 3, taked: 14.614, train_loss: 0.3430439623112374, train_acc: 0.8749, test_loss: 0.42189398445188997, test_acc: 0.8604
ep: 4, taked: 14.152, train_loss: 0.33348234302185953, train_acc: 0.8781, test_loss: 0.4137972265481949, test_acc: 0.8612
ep: 5, taked: 14.195, train_loss: 0.31623736220471405, train_acc: 0.8839166666666667, test_loss: 0.38468071073293686, test_acc: 0.8686
ep: 6, taked: 14.166, train_loss: 0.31094731092453004, train_acc: 0.8845166666666666, test_loss: 0.42263117507100106, test_acc: 0.8596
ep: 7, taked: 14.781, train_loss: 0.3063214588038465, train_acc: 0.8864666666666666, test_loss: 0.433824345

Добавим dropout и batchnorm-слои

In [35]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 2560),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(2560, 1280),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(1280, 640),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(640, 10)
)

In [36]:
trainer = torch.optim.Adam(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 61.211, train_loss: 1.1554567273626937, train_acc: 0.6349166666666667, test_loss: 0.6922964043915272, test_acc: 0.715
ep: 1, taked: 60.910, train_loss: 0.8973515784486811, train_acc: 0.6634833333333333, test_loss: 0.6768616020679474, test_acc: 0.7182
ep: 2, taked: 65.950, train_loss: 0.9332675200827578, train_acc: 0.6589166666666667, test_loss: 0.6716974593698979, test_acc: 0.7245
ep: 3, taked: 90.986, train_loss: 0.9684767200591716, train_acc: 0.6475166666666666, test_loss: 0.7056689485907555, test_acc: 0.7163
ep: 4, taked: 84.192, train_loss: 0.9934180931842074, train_acc: 0.6407333333333334, test_loss: 0.8257804855704307, test_acc: 0.7063
ep: 5, taked: 83.453, train_loss: 0.9919665374654405, train_acc: 0.6429166666666667, test_loss: 0.7131932653486729, test_acc: 0.7215
ep: 6, taked: 84.525, train_loss: 0.9617666972444413, train_acc: 0.64785, test_loss: 0.6987582288682461, test_acc: 0.7254
ep: 7, taked: 84.607, train_loss: 0.9633284310077099, train_acc: 0.64756666666666

In [44]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(512),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(256),
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(128),
    torch.nn.Linear(128, 10)
)

In [45]:
# попробуем уменьшить learning rate и добавим параметр betas
trainer = torch.optim.Adam(model.parameters(), lr=.0001, betas=(0.9, 0.999))
train_model()

ep: 0, taked: 13.909, train_loss: 0.6131979153511372, train_acc: 0.8171666666666667, test_loss: 0.43979710191488264, test_acc: 0.8513
ep: 1, taked: 13.288, train_loss: 0.36575534001309823, train_acc: 0.8770166666666667, test_loss: 0.3824111096560955, test_acc: 0.862
ep: 2, taked: 12.638, train_loss: 0.31026010291373474, train_acc: 0.8936166666666666, test_loss: 0.3617993209511042, test_acc: 0.8694
ep: 3, taked: 12.673, train_loss: 0.27332230356145415, train_acc: 0.9050166666666667, test_loss: 0.3505463063716888, test_acc: 0.8718
ep: 4, taked: 12.704, train_loss: 0.24554826414331477, train_acc: 0.9146166666666666, test_loss: 0.3417630882933736, test_acc: 0.8751
ep: 5, taked: 12.800, train_loss: 0.22141029866451914, train_acc: 0.92275, test_loss: 0.33260042034089565, test_acc: 0.8794
ep: 6, taked: 12.838, train_loss: 0.19945057037662953, train_acc: 0.931, test_loss: 0.34698844589293004, test_acc: 0.8749
ep: 7, taked: 12.767, train_loss: 0.1796953554800216, train_acc: 0.9388166666666666, 

Выводы:
1. Построена и обучена модель многослойной НС
2. Модель была оптимизирована несколькими различными способами, получив тем самым точность на тестовой выборке больше пороговой (88%)